In [1]:
########## Format ERA5 data and write to a zarr file #################

In [2]:
import numpy as np
import xarray as xr
import pandas as pd
from distributed import Client
import dask_jobqueue
import matplotlib.pyplot as plt
import glob
import netCDF4 as nc
import zarr
import nc_time_axis
import xskillscore as xs
import cartopy as cart
from scipy.stats import skew
import xskillscore as xs
from scipy.stats import skewnorm
from scipy.special import erfinv, erf
from sklearn.metrics import mean_squared_error
np.set_printoptions(threshold=np.inf)

In [3]:
######################
cesm2_path       = '/global/scratch/users/harsha/LENS/cesm2/tasmax/'
cvals            = '/global/scratch/users/harsha/LENS/cesm2/cvals/'
cvals1           = '/global/scratch/users/harsha/LENS/cesm2/tmax_mem'
era5_path        = '/global/scratch/users/harsha/ERA5/tasmax/'
era5_noleap_path = '/global/scratch/users/harsha/ERA5/tasmax/era5_noleap_1950_2021.zarr'
##############################
print(era5_noleap_path)

/global/scratch/users/harsha/ERA5/tasmax/era5_noleap_1950_2021.zarr


In [4]:
job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
#job_extra =['--qos=lr6_lowprio','--account=ac_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=lr_lowprio','--account=ac_cumulus']
cluster = dask_jobqueue.SLURMCluster(queue="cf1", cores=10, walltime='6:00:00', 
                local_directory='/global/scratch/users/harsha/dask_space/', 
                log_directory='/global/scratch/users/harsha/dask_space/', 
                job_extra_directives=job_extra, interface='eth0', memory="192GB") 
client  = Client(cluster) 
cluster.scale_up(4)

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34027 instead
  warnings.warn(


In [5]:
cluster

In [6]:
def to_daily(ds):
    year = ds.days.dt.year
    day  = ds.days.dt.dayofyear
    # assign new coords
    ds   = ds.assign_coords(year=("days", year.data), day=("days", day.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(days=("year", "day")).unstack("days")

In [7]:
tera = xr.open_mfdataset(era5_path+'*.nc',concat_dim='days',combine='nested')
tera = tera.tasmax
tera

<xarray.DataArray 'tasmax' (days: 26298, lats: 105, lons: 237)>
dask.array<concatenate, shape=(26298, 105, 237), dtype=float64, chunksize=(4018, 105, 237), chunktype=numpy.ndarray>
Coordinates:
  * lats     (lats) float64 50.0 49.75 49.5 49.25 49.0 ... 24.75 24.5 24.25 24.0
  * lons     (lons) float64 235.0 235.2 235.5 235.8 ... 293.2 293.5 293.8 294.0
Dimensions without coordinates: days

In [8]:
#Convert era5's 'days' coordinate to a datetime.
tera['days'] = pd.to_datetime(tera['days'], origin='1950-01-01', unit='D')
tera

<xarray.DataArray 'tasmax' (days: 26298, lats: 105, lons: 237)>
dask.array<concatenate, shape=(26298, 105, 237), dtype=float64, chunksize=(4018, 105, 237), chunktype=numpy.ndarray>
Coordinates:
  * lats     (lats) float64 50.0 49.75 49.5 49.25 49.0 ... 24.75 24.5 24.25 24.0
  * lons     (lons) float64 235.0 235.2 235.5 235.8 ... 293.2 293.5 293.8 294.0
  * days     (days) datetime64[ns] 1950-01-01 1950-01-02 ... 2021-12-31

In [9]:
# leap_days       = ((tera['days'].dt.is_leap_year) & (tera['days'].dt.month == 2) & (tera['days'].dt.day == 29))
# #Leap days is a boolean array. Firat, check if we got the correct leap days.
# leap_days_dates = tera['days'].where(leap_days, drop=True)
# leap_days_dates

In [10]:
# #Now, Let use leap_days to create the complement boolean array and ue it to drop leap_days.
# not_leap_days = ~leap_days
# tera_noleap   =  tera.where(not_leap_days,drop= True)
# tera_noleap

In [11]:
%%time
#d = daily
tera_d = to_daily(tera)
tera_d

CPU times: user 55.4 ms, sys: 9.07 ms, total: 64.5 ms
Wall time: 64.1 ms


/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.r

<xarray.DataArray 'tasmax' (lats: 105, lons: 237, year: 72, day: 366)>
dask.array<reshape, shape=(105, 237, 72, 366), dtype=float64, chunksize=(105, 237, 11, 366), chunktype=numpy.ndarray>
Coordinates:
  * year     (year) int64 1950 1951 1952 1953 1954 ... 2017 2018 2019 2020 2021
  * day      (day) int64 1 2 3 4 5 6 7 8 9 ... 359 360 361 362 363 364 365 366
  * lats     (lats) float64 50.0 49.75 49.5 49.25 49.0 ... 24.75 24.5 24.25 24.0
  * lons     (lons) float64 235.0 235.2 235.5 235.8 ... 293.2 293.5 293.8 294.0

In [12]:
noleap_days  = np.arange(365)+1
tera_dnoleap = tera_d.sel(day = noleap_days, drop=True)
tera_dnoleap

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.DataArray 'tasmax' (lats: 105, lons: 237, year: 72, day: 365)>
dask.array<getitem, shape=(105, 237, 72, 365), dtype=float64, chunksize=(105, 237, 11, 365), chunktype=numpy.ndarray>
Coordinates:
  * year     (year) int64 1950 1951 1952 1953 1954 ... 2017 2018 2019 2020 2021
  * day      (day) int64 1 2 3 4 5 6 7 8 9 ... 358 359 360 361 362 363 364 365
  * lats     (lats) float64 50.0 49.75 49.5 49.25 49.0 ... 24.75 24.5 24.25 24.0
  * lons     (lons) float64 235.0 235.2 235.5 235.8 ... 293.2 293.5 293.8 294.0

In [13]:
%%time
######## Save the data as a zarr file ########
tera_dnoleap = tera_dnoleap.chunk({'year':10})
#tera_dnoleap.to_dataset().to_zarr(era5_noleap_path,mode='w')

CPU times: user 691 µs, sys: 953 µs, total: 1.64 ms
Wall time: 1.6 ms


In [14]:
tera_dnoleap = xr.open_zarr(era5_noleap_path)
tera_dnoleap

<xarray.Dataset>
Dimensions:  (day: 365, lats: 105, lons: 237, year: 72)
Coordinates:
  * day      (day) int64 1 2 3 4 5 6 7 8 9 ... 358 359 360 361 362 363 364 365
  * lats     (lats) float64 50.0 49.75 49.5 49.25 49.0 ... 24.75 24.5 24.25 24.0
  * lons     (lons) float64 235.0 235.2 235.5 235.8 ... 293.2 293.5 293.8 294.0
  * year     (year) int64 1950 1951 1952 1953 1954 ... 2017 2018 2019 2020 2021
Data variables:
    tasmax   (lats, lons, year, day) float64 dask.array<chunksize=(105, 237, 10, 365), meta=np.ndarray>